In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
import pandas as pd
import numpy as np
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'
import time
import matplotlib.pyplot as plt
import cv2
import seaborn as sns
sns.set_style('darkgrid')
import shutil
from sklearn.metrics import confusion_matrix, classification_report
from sklearn.model_selection import train_test_split
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dense, Activation,Dropout,Conv2D, MaxPooling2D,BatchNormalization
from tensorflow.keras.optimizers import Adam, Adamax
from tensorflow.keras.metrics import categorical_crossentropy
from tensorflow.keras import regularizers
from tensorflow.keras.models import Model
from tensorflow.keras import backend as K
import time
from tqdm import tqdm
from PIL import Image
from google.colab.patches import cv2_imshow

In [3]:
# Load the model architecture without loading the optimizer state
model = tf.keras.models.load_model("/content/drive/MyDrive/Code/Efficientnetv2S_89.35.keras", compile=False)

# Load the weights
model.load_weights("/content/drive/MyDrive/Code/Efficientnetv2S_weights89.35.weights.h5")

# Recompile the model with the Adam optimizer (or any other optimizer you need)
model.compile(optimizer='adam', loss='categorical_crossentropy')  # Adjust the loss and metrics based on your use case

In [4]:
# # def grad_cam_plus(model, img,
# #                   layer_name="top_conv", label_name=None,
# #                   category_id=None):
# #     """Get a heatmap by Grad-CAM++.

# #     Args:
# #         model: A model object, build from tf.keras 2.X.
# #         img: An image ndarray.
# #         layer_name: A string, layer name in model.
# #         label_name: A list or None,
# #             show the label name by assign this argument,
# #             it should be a list of all label names.
# #         category_id: An integer, index of the class.
# #             Default is the category with the highest score in the prediction.

# #     Return:
# #         A heatmap ndarray(without color).
# #     """
# #     img_tensor = np.expand_dims(img, axis=0)

# #     conv_layer = model.get_layer(layer_name)
# #     heatmap_model = Model([model.inputs], [conv_layer.output, model.output])

# #     with tf.GradientTape() as gtape1:
# #         with tf.GradientTape() as gtape2:
# #             with tf.GradientTape() as gtape3:
# #                 conv_output, predictions = heatmap_model(img_tensor)
# #                 if category_id is None:
# #                     category_id = np.argmax(predictions[0])
# #                 if label_name is not None:
# #                     print(label_name[category_id])
# #                 output = predictions[:, category_id]
# #                 conv_first_grad = gtape3.gradient(output, conv_output)
# #             conv_second_grad = gtape2.gradient(conv_first_grad, conv_output)
# #         conv_third_grad = gtape1.gradient(conv_second_grad, conv_output)

# #     global_sum = np.sum(conv_output, axis=(0, 1, 2))

# #     alpha_num = conv_second_grad[0]
# #     alpha_denom = conv_second_grad[0]*2.0 + conv_third_grad[0]*global_sum
# #     alpha_denom = np.where(alpha_denom != 0.0, alpha_denom, 1e-10)

# #     alphas = alpha_num/alpha_denom
# #     alpha_normalization_constant = np.sum(alphas, axis=(0,1))
# #     alphas /= alpha_normalization_constant

# #     weights = np.maximum(conv_first_grad[0], 0.0)

# #     deep_linearization_weights = np.sum(weights*alphas, axis=(0,1))
# #     grad_cam_map = np.sum(deep_linearization_weights*conv_output[0], axis=2)

# #     heatmap = np.maximum(grad_cam_map, 0)
# #     max_heat = np.max(heatmap)
# #     if max_heat == 0:
# #         max_heat = 1e-10
# #     heatmap /= max_heat

# #     return heatmap

# def grad_cam_plus(model, img,
#                   layer_name="top_conv", label_name=None,
#                   category_id=None):
#     """Get a heatmap by Grad-CAM++.

#     Args:
#         model: A model object, build from tf.keras 2.X.
#         img: An image ndarray.
#         layer_name: A string, layer name in model.
#         label_name: A list or None,
#             show the label name by assign this argument,
#             it should be a list of all label names.
#         category_id: An integer, index of the class.
#             Default is the category with the highest score in the prediction.

#     Return:
#         A heatmap ndarray(without color).
#     """
#     img_tensor = np.expand_dims(img, axis=0)

#     conv_layer = model.get_layer(layer_name)
#     heatmap_model = Model([model.inputs], [conv_layer.output, model.output])

#     with tf.GradientTape() as gtape1:
#         with tf.GradientTape() as gtape2:
#             with tf.GradientTape() as gtape3:
#                 conv_output, predictions = heatmap_model(img_tensor)
#                 if category_id is None:
#                     category_id = np.argmax(predictions[0]) # Assuming predictions is a list of lists
#                 if label_name is not None:
#                     print(label_name[category_id])
#                 # Changing list indexing: predictions[0] assumes your predictions is list of lists
#                 output = predictions[0][category_id]  # Access element correctly within list
#                 conv_first_grad = gtape3.gradient(output, conv_output)
#             conv_second_grad = gtape2.gradient(conv_first_grad, conv_output)
#         conv_third_grad = gtape1.gradient(conv_second_grad, conv_output)

#     global_sum = np.sum(conv_output, axis=(0, 1, 2))

#     alpha_num = conv_second_grad[0]
#     alpha_denom = conv_second_grad[0]*2.0 + conv_third_grad[0]*global_sum
#     alpha_denom = np.where(alpha_denom != 0.0, alpha_denom, 1e-10)

#     alphas = alpha_num/alpha_denom
#     alpha_normalization_constant = np.sum(alphas, axis=(0,1))
#     alphas /= alpha_normalization_constant

#     weights = np.maximum(conv_first_grad[0], 0.0)

#     deep_linearization_weights = np.sum(weights*alphas, axis=(0,1))
#     grad_cam_map = np.sum(deep_linearization_weights*conv_output[0], axis=2)

#     heatmap = np.maximum(grad_cam_map, 0)
#     max_heat = np.max(heatmap)
#     if max_heat == 0:
#         max_heat = 1e-10
#     heatmap /= max_heat

#     return heatmap

def grad_cam_plus(model, img,
                  layer_name="top_conv", label_name=None,
                  category_id=None):
    """Get a heatmap by Grad-CAM++.

    Args:
        model: A model object, build from tf.keras 2.X.
        img: An image ndarray.
        layer_name: A string, layer name in model.
        label_name: A list or None,
            show the label name by assign this argument,
            it should be a list of all label names.
        category_id: An integer, index of the class.
            Default is the category with the highest score in the prediction.

    Return:
        A heatmap ndarray(without color).
    """
    img_tensor = np.expand_dims(img, axis=0)

    conv_layer = model.get_layer(layer_name)
    heatmap_model = Model([model.inputs], [conv_layer.output, model.output])

    with tf.GradientTape() as gtape1:
        with tf.GradientTape() as gtape2:
            with tf.GradientTape() as gtape3:
                conv_output, predictions = heatmap_model(img_tensor)
                if category_id is None:
                    category_id = np.argmax(predictions[0]) # Assuming predictions is a list of lists

                    # Check if category_id is within bounds and adjust if necessary
                    num_classes = predictions[0].shape[0]  # Get the number of classes from the prediction shape
                    if category_id >= num_classes:
                        category_id = num_classes - 1  # Adjust to the maximum valid index

                if label_name is not None:
                    print(label_name[category_id])

                # Changing list indexing: predictions[0] assumes your predictions is list of lists
                output = predictions[0][category_id]  # Access element correctly within list

                conv_first_grad = gtape3.gradient(output, conv_output)
            conv_second_grad = gtape2.gradient(conv_first_grad, conv_output)
        conv_third_grad = gtape1.gradient(conv_second_grad, conv_output)

    global_sum = np.sum(conv_output, axis=(0, 1, 2))

    alpha_num = conv_second_grad[0]
    alpha_denom = conv_second_grad[0]*2.0 + conv_third_grad[0]*global_sum
    alpha_denom = np.where(alpha_denom != 0.0, alpha_denom, 1e-10)

    alphas = alpha_num/alpha_denom
    alpha_normalization_constant = np.sum(alphas, axis=(0,1))
    alphas /= alpha_normalization_constant

    weights = np.maximum(conv_first_grad[0], 0.0)

    deep_linearization_weights = np.sum(weights*alphas, axis=(0,1))
    grad_cam_map = np.sum(deep_linearization_weights*conv_output[0], axis=2)

    heatmap = np.maximum(grad_cam_map, 0)
    max_heat = np.max(heatmap)
    if max_heat == 0:
        max_heat = 1e-10
    heatmap /= max_heat

    return heatmap

In [5]:
def show_imgwithheat(img_path, heatmap, alpha=0.8, return_array=False):
    """Show the image with heatmap.

    Args:
        img_path: string.
        heatmap: image array, get it by calling grad_cam().
        alpha: float, transparency of heatmap.
        return_array: bool, return a superimposed image array or not.
    Return:
        None or image array.
    """
    img = cv2.imread(img_path)
    heatmap = cv2.resize(heatmap, (img.shape[1], img.shape[0]))
    heatmap = (heatmap*255).astype("uint8")
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = heatmap * alpha + img
    superimposed_img = np.clip(superimposed_img, 0, 255).astype("uint8")
    superimposed_img = cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB)

    imgwithheat = Image.fromarray(superimposed_img)

    return superimposed_img

def load_and_preprocess_image(img_path, target_size=(224, 224)):
    img = cv2.imread(img_path)
    img = cv2.resize(img, target_size)
    img_tensor = tf.keras.applications.vgg16.preprocess_input(np.expand_dims(img, axis=0))
    return img, img_tensor

In [6]:
sdir = r"/content/drive/MyDrive/Code/new_images/spectogram_images/siren"  # Parent directory containing subfolders
ddir = r"/content/drive/MyDrive/Code/att_img/spectogram_pics/siren"  # Parent directory to save the output

In [7]:
for filename in os.listdir(sdir):
    if filename.endswith(('.jpg', '.png', '.jpeg')):
        img_path = os.path.join(sdir, filename)
        image = cv2.imread(img_path)
        image = cv2.resize(image, (224,224))
        heatmap_plus=grad_cam_plus(model, image, label_name=None, category_id=None)
        # Overlay heatmap on the image
        image = show_imgwithheat(img_path, heatmap_plus)
        print("X")
        # Save the result to the output folder
        output_path = os.path.join(ddir, f"{filename}")
        cv2.imwrite(output_path, image)

X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
X
